In [143]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [144]:
import re

def attributes(str1):
    exp = '\([^)]+\)'
    match = re.findall(exp, str1)
    return [m for m in str(match) if m.isalpha()]

def predicates(str1):
    exp = '[a-z~]+\([A-Za-z,]+\)'
    return re.findall(exp, str1)

def deMorgan(sentence):
    str1 = ''.join(list(sentence).copy())
    str1 = str1.replace('~~','')
    flag = '[' in str1
    str1 = str1.replace('~[','')
    str1 = str1.strip(']')
    for predicate in predicates(str1):
        str1 = str1.replace(predicate, f'~{predicate}')
    s = list(str1)
    for i, c in enumerate(str1):
        if c == '|':
            s[i] = '&'
        elif c == '&':
            s[i] = '|'
    str1 = ''.join(s)    
    str1 = str1.replace('~~','')
    return f'[{str1}]' if flag else str1

In [145]:
def skolemization(sentence):
    SKOLEM_CONSTANTS = [f'{chr(c)}' for c in range(ord('A'), ord('Z')+1)]
    statement = ''.join(list(sentence).copy())
    matches = re.findall('[∀∃].', statement)
    for match in matches[::-1]:
        statement = statement.replace(match, '')
        statements = re.findall('\[\[[^]]+\]]', statement)
        for s in statements:
            statement = statement.replace(s, s[1:-1])
        for predicate in predicates(statement):
            attribute = attributes(predicate)
            if ''.join(attribute).islower():
                statement = statement.replace(match[1],SKOLEM_CONSTANTS.pop(0))
            else:
                aU = [a for a in attribute if not a.islower()][0]
                statement = statement.replace(aU, f'{SKOLEM_CONSTANTS.pop(0)}({match[1]})')
    return statement

In [146]:
import re

def foltocnf(fol):    
    statement = fol.replace("<=>", "_")
    while '_' in statement:
        i = statement.index('_')
        new_statement = '[' + statement[:i] + '=>' + statement[i+1:] + '] & ['+ statement[i+1:] + '=>' + statement[:i] + ']'
        statement = new_statement
    statement = statement.replace("=>", "-")
    exp = '\[([^]]+)\]'
    statements = re.findall(exp, statement)
    for i, s in enumerate(statements):
        if '[' in s and ']' not in s:
            statements[i] += ']'
    for s in statements:
        statement = statement.replace(s, foltocnf(s))
    while '-' in statement:
        i = statement.index('-')
        br = statement.index('[') if '[' in statement else 0
        new_statement = '~' + statement[br:i] + ' | ' + statement[i+1:]
        statement = statement[:br] + new_statement if br > 0 else new_statement
    while '~∀' in statement:
        i = statement.index('~∀')
        statement = list(statement)
        statement[i], statement[i+1], statement[i+2] = '∃', statement[i+2], '~'
        statement = ''.join(statement)
    while '~∃' in statement:
        i = statement.index('~∃')
        s = list(statement)
        s[i], s[i+1], s[i+2] = '∀', s[i+2], '~'
        statement = ''.join(s)
    statement = statement.replace('~[∀','[~∀')
    statement = statement.replace('~[∃','[~∃')
    exp = '(~[∀|∃].)'
    statements = re.findall(exp, statement)
    for s in statements:
        statement = statement.replace(s, foltocnf(s))
    exp = '~\[[^]]+\]'
    statements = re.findall(exp, statement)
    for s in statements:
        statement = statement.replace(s, deMorgan(s))
    return statement

In [147]:
def main():
    print("Enter the FOL statement (Give spaces between operators)")
    fol = input()
    print("The given FOL statement in CNF is:")
    print(skolemization(foltocnf(fol)))

In [148]:
#Testcase - 1
main()

Enter the FOL statement (Give spaces between operators)
animal(y)<=>loves(x,y)
The given FOL statement in CNF is:
[~animal(y) | loves(x,y)] & [~loves(x,y) | animal(y)]


In [149]:
#Testcase - 2
main()

Enter the FOL statement (Give spaces between operators)
[american(x) & weapon(y) & sells(x,y,z) & hostile(z)] => criminal(x)
The given FOL statement in CNF is:
[~american(x) | ~weapon(y) | ~sells(x,y,z) | ~hostile(z)]  |  criminal(x)


In [150]:
#Testcase - 3
main()

Enter the FOL statement (Give spaces between operators)
∀x[∀y[animal(y) => loves(x,y)]] => [∃z[loves(z,x)]]
The given FOL statement in CNF is:
[animal(G(x))  &  ~loves(x,G(x))]  |  [loves(F(y),x)]
